In [1]:
import os
import sys
import json
from iconic_tools.langchain import InstructSonnet, InstructOpus3, InstructGPT4, InstructGeminiPro, InstructGPT35
from langchain_core.prompts import ChatPromptTemplate

/Users/murrayshanahan/iconic_dir/digital_actor_benchmark/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CONSTANTS AND INITIALISATION

PATH = os.path.abspath(os.getcwd())

# DIALOGUE_MODEL = InstructGeminiPro(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGeminiPro(temperature=1.0, max_tokens=3000)

# DIALOGUE_MODEL = InstructGPT4(temperature=1.0, max_tokens=3000)
# QUERY_MODEL = InstructGPT4(temperature=1.0, max_tokens=3000)

DIALOGUE_MODEL = InstructSonnet(temperature=1.0, max_tokens=3000)
QUERY_MODEL = InstructSonnet(temperature=1.0, max_tokens=3000)

# GAME = "troy"
# SCENE = "achilles"
# ACTORS = ["Achilles", "Odysseus"]

# GAME = "amadeus"
# SCENE = "constanze"
# ACTORS = ["Mozart", "Constanze", "Schikaneder"]

# GAME = "cedar_rapids"
# SCENE = "car"
# ACTORS = ["Tim", "Ronald", "Dean", "Joan"]

GAME = "bound"
SCENE = "coffee"
ACTORS = ["Violet", "Corky"]

RED = "\033[91m"
GREEN = "\033[92m"
BLUE = "\033[94m"
YELLOW = "\033[93m"
WHITE = "\033[0m"

In [3]:
# UTILITIES


def load_prompt(filename):
    with open(PATH + f"/prompts/{filename}") as f:
        return f.read()


def write_transcript(dialogue, filename):
    with open(PATH + f"/transcripts/{filename}", "w") as f:
        f.write(dialogue)


def list_to_conjunction(L):
    """Takes a list strings and returns a string with every element in the list separated by commas."""
    if L == "":
        return ""
    elif len(L) == 1:
        return L[0]
    elif len(L) == 2:
        return f"{L[0]} and {L[1]}"
    else:
        return ", ".join(L[:-1]) + f", and {L[-1]}"

In [4]:
# PROMPT TEMPLATES AND INSTRUCTION PROMPTS


dialogue_instruction_prefix = """
You are going to generate one line of dialogue for a scene in the middle of a computer game.
"""


preamble_template = """
{instruction_prefix}
This is the game back story. {back_story}\n
Here is a description of the scene in question. {scene_description}{scene_supplement}\n
The characters in the dialogue are {actors}.
"""

instruction_template = """
{preamble}
Here is the dialogue so far\n\n
{dialogue}
{instruction_suffix}
"""

speech_template = '[{actor}]: {speech}\n'

dialogue_instruction_suffix = """
Give me the next line in the dialogue in the same format. Don't provide stage directions, just the actor's words.\n
"""

query_instruction_suffix_template = """
{query} Answer with a single word, yes or no.
"""

query_instruction_prefix = """
You are going to answer a single yes/no question about the current state of the dialogue in a scene in the middle of a computer game.\n
"""

In [5]:
# BUILDING DIALOGUES


def prompt_llm(prompt, model):
    # print(prompt)
    # print()
    prompt = ChatPromptTemplate.from_template(template=prompt)
    chain = prompt | model
    return chain


def load_prompts(scene_version, supplement_version=-1):
    back_story = load_prompt(GAME + "/back_story.txt")
    scene_description = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_scene_description_" + str(scene_version) + ".txt")
    if supplement_version == -1:  # no supplementary scene text
        scene_supplement = ""
    else:
        scene_supplement = "\n\n" + load_prompt(
            GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_scene_supplement_" + str(scene_version) + ".txt")
    opening_speech = load_prompt(
        GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_opening_speech.txt")
    query = load_prompt(GAME + "/scenes/" + SCENE + "_scene/" + SCENE + "_query1.txt")
    end_queries = [query]
    return (back_story, scene_description, scene_supplement, opening_speech, end_queries)


def query_dialogue(
        dialogue, back_story, scene_description,
        actors, query):
    query_model = QUERY_MODEL
    query_preamble = preamble_template.format(
        instruction_prefix=query_instruction_prefix,
        back_story=back_story,
        scene_description=scene_description,
        scene_supplement="",
        actors=list_to_conjunction(actors))
    query_instruction_suffix = query_instruction_suffix_template.format(query=query)
    query_test = instruction_template.format(
        preamble=query_preamble, dialogue=dialogue,
        instruction_suffix=query_instruction_suffix)
    chain = prompt_llm(query_test, query_model)
    response = chain.invoke({})
    return response


def sim_mini_scene(
        back_story, scene_description, scene_supplement,
        actors, opening_speech, end_queries,
        player=False,
        max_turns=10):
    """Generates dialogue for a mini-scene.
    
    back_story: The backdrop to the whole game.
    scene_description: A description of this particular mini-scene. Should state the goals of the scene.
    scene_supplement: Extra text describing the scene. Can be used to simulate adversarial players, for example.
    actors: A list of the names of the actors involved in generating dialogue.
    opening_speech: The first words spoken by actor1. (These have to be scripted.)
    player: If True then the user is one of the players, otherwise both players are LLMs.
    end_queries: A list of natural language questions; the scene is terminated if all answers are yes.
    max_turns: The scene will terminate when this many turns have been taken whether or not goals have been reached.
    """

    print(RED + scene_description)
    print()

    dialogue_model = DIALOGUE_MODEL
 
    dialogue_preamble = preamble_template.format(
        instruction_prefix=dialogue_instruction_prefix,
        back_story=back_story,
        scene_description=scene_description,
        scene_supplement=scene_supplement,
        actors=list_to_conjunction(actors))
    
    if player:
        speech = input()
        response = speech_template.format(actor=actors[0], speech=speech)
    else:
        response = speech_template.format(actor=actors[0], speech=opening_speech)

    dialogue = response + "\n"

    print(GREEN + response)

    turn = 1
    success = False

    while turn < max_turns and not success:
        
        if player and (turn % 2 == 0):  # user is playing actor 1
            speech = input()
            response = speech_template.format(actor=actors[0], speech=speech)
        else:  # both actors are played by the LLM
            prompt = instruction_template.format(
                preamble=dialogue_preamble, dialogue=dialogue,
                instruction_suffix=dialogue_instruction_suffix)
            chain = prompt_llm(prompt, dialogue_model)
            response = chain.invoke({}) + "\n"

        dialogue += response

        print(GREEN + response)

        # Have the conditions for ending the scene been met?
        success = True
        for  query in end_queries:
            response = query_dialogue(
                dialogue, back_story, scene_description,
                actors, query)
            success = success and (response[0:3] == "Yes" or response[0:3] == "yes")
            # print("Exit query response: {}".format(response))
            # print()

        turn += 1
    
    return dialogue

In [6]:
# # GENERATE DIALOGUES

# player = False
# supplement_version = -1
# n_versions = 3
# n_runs = 3

# for scene_version in range(n_versions):
#     print(WHITE + "VERSION {}".format(scene_version))
#     print()
#     print()
#     for run_no in range(n_runs):
#         print(WHITE + "Run {}".format(run_no))
#         print()

#         (back_story, scene_description, scene_supplement,
#          opening_speech, end_queries) = load_prompts(scene_version, supplement_version)
#         dialogue = sim_mini_scene(
#             back_story, scene_description, scene_supplement,
#             ACTORS, opening_speech, end_queries,
#             player=player, max_turns=20)
        
#         query = end_queries[0]
#         response = query_dialogue(
#             dialogue, back_story, scene_description,
#             ACTORS, query)
        
#         print(BLUE + query + ' ' + response)
#         print()
#         print()

#         write_transcript(dialogue, GAME + "/transcript_" + str(scene_version) + "_" + str(run_no) + ".txt")

In [7]:
player = False
scene_version = 1
supplement_version = -1

(back_story, scene_description, scene_supplement,
 opening_speech, end_queries) = load_prompts(scene_version, supplement_version)
dialogue = sim_mini_scene(
    back_story, scene_description, scene_supplement,
    ACTORS, opening_speech, end_queries,
    player=player, max_turns=20)

Corky, in her role as the new plumber and painter for the block, has just moved into an apartment on the same floor as Violet. Shortly before this scene, they met for the first time in the elevator, where they exchanged pleasantries and a couple of glances. Now Violet has turned up at Corky's door. The dialogue includes all the following elements, but as a skilled screenwriter you can weave them together seamlessly, ensuring the transitions from one topic of conversation are gradual, smooth, and natural. Violet likes the look of Corky and wants to get to know her, so she has brought her a cup of coffee. In a gentle way, she complains about Corky using noisy power tools early in the morning. (Violet is a night person.) Corky promises to hold off with the noise until a little later in the day. But Violet doesn't want to leave yet, so she questions Corky about her work, and, playing the part of a 'helpless female', tells her how impressed she is. The conversation continues, with Violet ad